In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import os

In [1]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from tqdm import tqdm, tqdm_pandas

import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
nltk.download('wordnet')
nltk.download('stopwords')

import pandas as pd
import os
import re
import numpy as np

import gensim

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alanliu99/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alanliu99/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
%%time
#os.chdir('../')
business = pd.read_csv('data/business.csv')
#review = pd.read_csv('test/testdata/test_review.csv')
las_vagas_reviews = pd.read_csv('data/Las_Vegas_review.csv')
#reviews = pd.read_csv('data/reviews.csv')

CPU times: user 11.7 s, sys: 1.89 s, total: 13.6 s
Wall time: 15.3 s


In [3]:
las_vagas_reviews.text= las_vagas_reviews.text.fillna("")
rest = las_vagas_reviews[las_vagas_reviews.text.apply(lambda x: len(x.split())) > 50]

In [4]:
all_id = np.unique(rest.business_id)
test_df = rest[rest.business_id.isin(all_id)].copy()

In [5]:
%%time
#Lemmatize, remove punctuations and symbols
lmtzr = WordNetLemmatizer()
tokenizer = TweetTokenizer()
sent_pre1 = test_df['text'].apply(lambda a: [lmtzr.lemmatize(word) for word in tokenizer.tokenize(a.lower())]) 

CPU times: user 18min 25s, sys: 1min 9s, total: 19min 34s
Wall time: 20min 12s


In [6]:
sent_pre2 = sent_pre1.apply(lambda a:[word for word in a if ((re.search(r'[^\w\s]', word) is None)|(len(word)>1))])

In [7]:
st_W = set([lmtzr.lemmatize(word) for word in stopwords.words('english')])
tokenised_sentences =  sent_pre2  

In [8]:
#Create Vocabulory
vocab = []
for sent in tokenised_sentences:
    for word in sent:
        vocab.append(word)
#Remove stopwords for LDA
vocab = set(vocab) - st_W

In [9]:
#Creating Bag of Words
vectorizer = CountVectorizer(binary=False
                             , vocabulary = list(vocab)
                             , min_df = 3, max_df=.8)
vectorizer = vectorizer.fit(tokenised_sentences.apply(lambda a: " ".join(a)))

BoW = vectorizer.transform(tokenised_sentences.apply(lambda a: " ".join(a)))

In [19]:
BoW

<998042x350666 sparse matrix of type '<class 'numpy.int64'>'
	with 59961102 stored elements in Compressed Sparse Row format>

In [ ]:
dictionary = gensim.corpora.Dictionary(tokenised_sentences)

In [14]:
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenised_sentences]

In [10]:
%%time
#Set number of LDA components
n_components = 30
#Number of words for showing LDA results
n_top_words = 20

#Helper function showing LDA topic words
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

#lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=n_components,
#                                       id2word=dictionary,
#                                       offset = 50,
#                                       random_state = 0,
#                                       passes=2,
#                                       workers=2)

CPU times: user 10 µs, sys: 83 µs, total: 93 µs
Wall time: 33.1 µs


In [11]:
#Run LDA on bag of words of review text
lda = LatentDirichletAllocation(n_components=n_components, max_iter=20,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [12]:
%%time
BoW_lda = lda.fit_transform(BoW) 

CPU times: user 1d 1h 48min 3s, sys: 1h 18min 19s, total: 1d 3h 6min 23s
Wall time: 21h 42min 11s


In [13]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda.score(BoW))

Log Likelihood:  -575588415.9834012


In [14]:
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda.perplexity(BoW))

Perplexity:  2737.087717121425


# Save the LDA model

In [20]:
import pickle

# Save to file in the current working directory
pkl_filename = "lda_30_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lda, file)

#with open('BoW_pkl.pkl', 'wb') as f:
#    pickle.dump(BoW,f)

In [18]:
# Load from file
with open('model/lda_model.pkl', 'rb') as file:
    lda_model = pickle.load(file)
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(BoW))

Perplexity:  28953856225.40085


In [27]:
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: room hotel casino stay strip floor stayed night nice bathroom day check area old clean parking front door healthy also
Topic #1: overpriced face buy suite smile actual bottle turkey passed daily pink deli win tub 1st pig market free bet watermelon
Topic #2: brunch shop evening shake margarita donut typical bf throughout visited mimosa con outdoor container bakery venetian pro meet wide luxor
Topic #3: breakfast egg pancake burrito hash 95 slider biscuit carne guacamole sausage omelet chipotle 99 stack fried corned chorizo scrambled lb
Topic #4: table minute order wait time came drink get line took around seated server 10 food waiting 30 service people long
Topic #5: taco mexican salsa morning nacho bean tortilla de asada la enchilada rice el quesadilla al corn street chicken pastor chile
Topic #6: bar beer music bartender game club night grab vibe beyond fee playing play loud cool additional girl sport dance cocktail
Topic #7: food like would time one back get place even serv

# Other Try

In [23]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda.score(BoW))

Log Likelihood:  -161246370.87889653


In [24]:
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda.perplexity(BoW))

Perplexity:  2183.234765622958


In [19]:
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: room bar vega stay night strip nice get floor stayed great view day area club check game one clean fun
Topic #1: pho additional paris planet vietnamese martini soooo refilled mark common broth macaron blood warned convenience jelly luggage grandma background 9pm
Topic #2: sushi roll ayce salmon chef rice japanese fresh sashimi poke crab nigiri earl nearly miso sake order uni yellowtail piece
Topic #3: patio gelato upgrade slider creme ravioli champagne romantic deli macaroon dj rio appetite brulee stew terrace marquee bath raised treasure
Topic #4: soup noodle thai broth pad chicken dumpling duck pita tom jean ube messy flan satay eggslut batali chive shelf kfc
Topic #5: cosmo de la el brie dan kimchi mai soho sora gordo monta mlife lo france mike cheek patience rum tonkotsu
Topic #6: cream dessert ice chocolate bar sweet space course roasted beat diner green lemon cooky red pulled apple fruit juice banana
Topic #7: burger cheese lobster sauce onion cooked potato ordered flav

In [20]:
# top worl for first 1000
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: macaron smoothie fruity pebble karaoke smoothy marrakech stingy pride eddie tiger samurai inevitable aradia mumbling airport damned billiards apologizing bobba
Topic #1: enchirrito clap pant death b4 moaning discovering builder committing minna shocker kissed ahve workplace ambrosia taboo insignificant khaki murder prrrrrrrrretty
Topic #2: mi banh bahn kinh de pupusas la que biet el dac en lee vietnam corazon carne xeo beaten pate viet
Topic #3: stock freshest philly jackpot carton maker shade pappy 1200 van winkle quickest lvcc cheesesteak 5000 mic tailored evil northeastern appetiser
Topic #4: ett courtney sig denna sux amarone kyra restaurangen alltså gick annunciate fajole panzenera dey paparadelle moonwalk rätt mer borde måttet
Topic #5: manzo di plin ai tajarin tuorli urbani brisson arrosto beppino jaden parmigiano marinate cray wined translated scoff cavoletti anatra sheryl
Topic #6: awesomeness chee jipped wirtz fleur foley hade koppa square asijah owed 4tonight tix s

In [14]:
# top words for first 50
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: chicken fry cheese waffle sauce fried bacon mac side ordered crispy potato tender slider frites bloody mary french buffalo got
Topic #1: buffet thai crab station leg curry lamb cosmo pad paris prime seafood com asian indian bouchon leftover yelp bellagio http
Topic #2: resort shower ravioli sub island gambling fully deli subway roti wheat eastern bun kiki poker banh moroccan dome hi treasure
Topic #3: room get place time one like hotel go vega would bar people even night food day drink stay line great
Topic #4: state tom flower horseradish shabu dealer gaming bath fuku bedroom poivre quote edo unassuming huevos amusement chelsea journey justin lodge
Topic #5: chicken rice pork fried ramen spicy noodle soup sauce beef bowl dish meat shrimp broth like chinese good roll bbq
Topic #6: tea circus filet boba milk ribeye mignon crap beet gift slut calorie dress unprofessional accompanied cuban vip monte taro facing
Topic #7: question health answered carnevino animal concerned code h